In [10]:
import urllib2, sys
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.core.display import HTML
import requests
import time
import datetime
import urlparse
import os

In [14]:
site = "https://www.hltv.org/stats/matches/performance/mapstatsid/52325/immortals-vs-cloud9"# performance page of match stats
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)
r = requests.get(site, headers=hdr)
tables = pd.read_html(r.text, header=0)
total_team_kda = tables[0] # total kills deaths and assists of team
who_kill_who = tables[1]  # who killed who
first_kills = tables[2]  #first kill of the round
awp_kills = tables[3]   #awp kills

In [11]:
site= "https://www.hltv.org/stats/matches/mapstatsid/52325/immortals-vs-cloud9" #overview page of match stats
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)

soup = BeautifulSoup(page)
  
match_time = soup.find_all("div", {"class":"small-text"})
for item in match_time:
    match_time = item.text
match_time = datetime.datetime.strptime(match_time, '%Y-%m-%d  %H:%MMap') #match date and time

r = requests.get(site, headers=hdr)
tables = pd.read_html(r.text, header=0)
team_a_stats, team_b_stats = tables[0], tables[1]

round_history_team = soup.find_all("div", class_ = "round-history-team-row") # winner of rounds and how rounds were won
round_history_team_a = round_history_team[0].find_all("img")
round_history_team_b = round_history_team[1].find_all("img")
team_a_scores = []
for scoreing in round_history_team_a:
     team_a_scores.append([ scoreing.get('title')])
team_b_scores = []                                                          #who won the round
for scoreing in round_history_team_b:
     team_b_scores.append([ scoreing.get('title')])
team_a_ending = []                                                          #how the round was won
for ending in round_history_team_a:
    url = urlparse.urlparse(ending.get('src'))
    base = os.path.basename(url.path)
    team_a_ending.append([os.path.splitext(base)[0]])   
team_b_ending = []
for ending in round_history_team_b:
    url = urlparse.urlparse(ending.get('src'))
    base = os.path.basename(url.path)
    team_b_ending.append([os.path.splitext(base)[0]])   



In [ ]:
site= "https://www.hltv.org/results?content=demo&team=7010&startDate=2016-09-01&endDate=2017-09-14"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [ ]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls1 = [result['href'] for result in results]

In [ ]:
site= "https://www.hltv.org/results?offset=100&startDate=2016-09-01&endDate=2017-09-14&content=demo&team=7010"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [ ]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls2 = [result['href'] for result in results]

In [ ]:
urls = urls1 + urls2

In [ ]:
def parse_match(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    site = 'https://www.hltv.org' + site
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    
   
    demo_url = 'https://www.hltv.org' + soup.find_all("a", class_="flexbox left-right-padding")[0]['href']
    
    try:
        vetos = soup.find_all("div", class_="standard-box veto-box")[1].find_all("div")[0].find_all("div")
        vetos = [veto.text for veto in vetos]
    except:
        vetos = None
        
    stats_url = 'https://www.hltv.org' + [a_element['href'] for a_element in soup.find_all("a") if a_element.text == "Detailed stats"][0]

    r = requests.get(site, headers=hdr)
    tables = pd.read_html(r.text, header=0)
    team_a, team_b = tables[0], tables[1]

    return {
        'url': site,
        'vetos': vetos,
        'stats_url': stats_url,
        'teams': [team_a, team_b]
    }

In [ ]:
matches = []
for idx, url in enumerate(urls):
    matches.append(parse_match(url))
    time.sleep(5)
    print 'match {0} done'.format(idx)